Hugging Face Example

> Fine tuning this model



Below  is used to check the status and utilization of NVIDIA GPUs on your system.

In [ ]:
!nvidia-smi

Fri Sep 12 12:59:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Below  is used to check whether you are using GPU or not

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Installed Packages

- **`transformers[sentencepiece]`**  
  Hugging Face’s `transformers` library for state-of-the-art NLP models (e.g., BERT, GPT, T5).  
  - The `[sentencepiece]` extra installs Google’s SentencePiece tokenizer, required for models like **mBART, MarianMT, and T5**.
  - mBART, MarianMT and T5 are pretrained NLP models

- **`datasets`**  
  Hugging Face’s dataset library.  
  - Provides access to thousands of NLP datasets.  
  - Useful for training and evaluating models.

- **`sacrebleu`**  
  A tool for calculating **BLEU scores**, commonly used in **machine translation evaluation**.

- **`rouge_score`**  
  For calculating **ROUGE metrics**, widely used in **summarization evaluation**.

- **`py7zr`**  
  A library to handle **7-zip (.7z) archive files**, which some datasets may be packaged in.

---

### Additional Flags

- **`-q`**  
  Quiet mode → suppresses most of the installation logs, keeping the output cleaner.


In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.6/413.6 kB 23.8 MB/s eta 0:00:00


**Accelerate** is a library from Hugging Face that makes it easy to train and run models on different hardware setups without changing your code.Automatically uses CPU, single GPU, or multiple GPUs (even TPUs) depending on what’s available. e.g If you go from 1 GPU → multi-GPU, you must rewrite with torch.nn.DataParallel or torch.nn.parallel.DistributedDataParallel.

In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.56.1
Uninstalling transformers-4.56.1:
  Successfully uninstalled transformers-4.56.1
Found existing installation: accelerate 1.10.1
Uninstalling accelerate-1.10.1:
  Successfully uninstalled accelerate-1.10.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 30.9 MB/s eta 0:00:00


***punkt*** is a pretrained tokenizer model that comes with NLTK (Natural Language Toolkit).It’s used mainly for sentence tokenization (splitting text into sentences).

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**Below is the example to use the Model**


### 📘 Explanation of Pegasus Summarization Workflow

1. **Importing the Model and Tokenizer**
   - `PegasusForConditionalGeneration` → Loads the **Pegasus model** designed for text summarization.  
   - `AutoTokenizer` → Loads the matching tokenizer to preprocess input text into token IDs.

2. **Loading a Pretrained Model**
   - `"google/pegasus-xsum"` → A pretrained Pegasus model fine-tuned on the **XSum dataset**, specialized for abstractive summarization (produces very concise summaries).

3. **Preparing the Input**
   - The input article is tokenized using the tokenizer.  
   - `max_length=1024` → Ensures the input text does not exceed the model’s maximum context size.  
   - `return_tensors="pt"` → Returns **PyTorch tensors** instead of plain Python lists, so the model can process them.

4. **Generating a Summary**
   - The model takes the tokenized input and generates a sequence of token IDs representing the summary.  
   - This step uses Pegasus’s **sequence-to-sequence generation** ability.

5. **Decoding the Output**
   - `tokenizer.batch_decode(...)` converts the generated token IDs back into readable text.  
   - `skip_special_tokens=True` → Removes tokens like `<pad>`, `<s>`, `</s>` that are used internally by the model.  
   - `clean_up_tokenization_spaces=False` → Prevents automatic changes to spacing, preserving the exact model output.

---

### ✅ In Short
This workflow takes an **article as input**, uses the **Pegasus pretrained summarization model**, and returns a **concise human-readable summary**.


In [ ]:
from transformers import AutoTokenizer, PegasusForConditionalGeneration

model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")

ARTICLE_TO_SUMMARIZE = (
    "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
    "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
    "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."
)
inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, return_tensors="pt")

# Generate Summary
summary_ids = model.generate(inputs["input_ids"])
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


"California's largest electricity provider has turned off power to hundreds of thousands of customers."

**Finetuning the Model with new datasets**
2. **Fine-Tuning (Optional)**
   - If your dataset is **different from news articles** (e.g., medical reports, legal documents, customer support logs), the model may not perform as well.  
   - In that case, you can **fine-tune** the model on your own dataset.  
   - Fine-tuning adapts the pretrained knowledge to your domain while reusing most of the original training.

3. **Training From Scratch (Not Recommended)**
   - Training Pegasus from zero would require **billions of text examples and huge compute resources** (TPUs/GPUs for weeks).  
   - Instead, we leverage pretrained models because they already “understand” language.


### 📘 Training Options for Models Like Pegasus

1. **Training From Scratch**
   - Start with **random weights** (model knows nothing).  
   - Train on a **massive corpus** (billions of sentences).  
   - Extremely costly (requires weeks on TPUs/GPUs).  
   - Example: when Google first trained Pegasus, they did this.

2. **Pretrained Model**
   - Already trained on a **huge general corpus** (Wikipedia, news, Common Crawl, etc.).  
   - Knows general language structure and semantics.  
   - Available on Hugging Face Hub (e.g., `google/pegasus-xsum`).  
   - Can be used **directly** or adapted.

3. **Fine-Tuning**
   - Start with a **pretrained model** (not random weights).  
   - Continue training on a **smaller, task-specific dataset** (e.g., medical articles, legal contracts).  
   - Much cheaper than scratch training.  
   - Usually the preferred method.

---

### ✅ In Short
- **Training from scratch** → No pretrained knowledge, weights start random.  
- **Using pretrained models** → Directly use them as they are.  
- **Fine-tuning pretrained models** → Adapt them to your dataset/domain.  


### 📘 What Are Weights in a Model?

1. **Definition**
   - Weights are **numerical values (parameters)** inside a neural network.  
   - They control how input data is transformed as it passes through the layers.  
   - During training, weights are adjusted to minimize the error (loss).

2. **How They Work**
   - Each connection between neurons has a weight.  
   - Input × Weight → passed through an activation function → output.  
   - Example: In a linear model, `y = wx + b`  
     - `w` = weight  
     - `b` = bias  

3. **In Pretrained Models**
   - A model like **Pegasus** has **hundreds of millions (or even billions) of weights**.  
   - Pretraining learns these weights on massive text datasets.  
   - When you download `google/pegasus-xsum`, you’re actually downloading the **weights file** (the learned knowledge).

4. **Fine-Tuning**
   - Instead of starting with random weights, you start with pretrained weights.  
   - Then update them slightly with your smaller dataset to adapt the model.

---

### ✅ In Short
- **Weights = the learned knowledge of the model.**  
- Pretrained models come with weights already trained on huge datasets.  
- Fine-tuning = slightly adjusting those weights for your task.  


### 📘 Explanation of Pegasus (CNN/DailyMail) Setup

1. **Importing Classes**
   - `AutoTokenizer` → Loads the appropriate tokenizer for the selected pretrained model.  
   - `AutoModelForSeq2SeqLM` → Loads a sequence-to-sequence model for tasks like summarization, translation, or text generation.

2. **Loading the Tokenizer**
   - `AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")`  
   - Downloads and initializes the tokenizer for **Pegasus trained on the CNN/DailyMail dataset**.  
   - This tokenizer converts raw text into input IDs and attention masks suitable for the model.

3. **Loading the Model**
   - `AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-cnn_dailymail")`  
   - Loads the pretrained Pegasus model fine-tuned on the **CNN/DailyMail summarization dataset**, which is designed to generate longer, more detailed summaries compared to `pegasus-xsum`.

4. **Moving the Model to Device**
   - `.to(device)` moves the model to the chosen hardware:
     - **CPU** → if no GPU is available.  
     - **GPU** → for faster training/inference (e.g., `device = torch.device("cuda")`).  

---

### ✅ In Short
This snippet sets up the **Pegasus CNN/DailyMail summarization model** along with its tokenizer, ensuring it is ready to run on the available hardware (CPU or GPU).


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")  #load a tokenizer

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-cnn_dailymail").to(device)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

### 📊 Difference Between `pegasus-xsum` and `pegasus-cnn_dailymail`

| Model                          | Dataset Used            | Summary Style            | Best Use Case |
|--------------------------------|-------------------------|--------------------------|---------------|
| **`pegasus-xsum`**             | XSum (Extreme Summarization) | Very short, highly **abstractive** (often 1–2 sentences, rephrased in new words). | When you want a **concise headline-style summary**. |
| **`pegasus-cnn_dailymail`**    | CNN/DailyMail           | Longer, more **extractive + abstractive** (typically 3–4 sentences, closer to original wording). | When you want a **detailed news-style summary**. |

---

### ✅ In Short
- Use **`pegasus-xsum`** → for **brief summaries** (like TL;DR headlines).  
- Use **`pegasus-cnn_dailymail`** → for **longer summaries** (like news digests).


In [ ]:
#dowload & unzip dataset

!wget https://github.com/krishnaik06/datasets/raw/refs/heads/main/summarizer-data.zip
!unzip summarizer-data.zip

--2025-09-12 13:09:00--  https://github.com/krishnaik06/datasets/raw/refs/heads/main/summarizer-data.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/krishnaik06/datasets/refs/heads/main/summarizer-data.zip [following]
--2025-09-12 13:09:01--  https://raw.githubusercontent.com/krishnaik06/datasets/refs/heads/main/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip’

summarizer-data.zip 100%[===================>]   7.54M  26.2MB/s    in 0.3s    

2025-09-12 13:09:06 (26.2 MB/s) - ‘summarizer-data.zip’ saved [79035

### 📘 Explanation of `load_from_disk` for Datasets

1. **Purpose**
   - `load_from_disk` is a function from Hugging Face `datasets`.  
   - It loads a **previously saved dataset** from disk into memory so it can be used for training, evaluation, or preprocessing.

2. **How It Works**
   - `'samsum_dataset'` → Path to the folder where the dataset was saved.  
   - The function restores the **dataset object** exactly as it was saved, including:
     - Features (columns, data types)
     - Train/test splits
     - Any preprocessing applied before saving

3. **Why It’s Useful**
   - Avoids **re-downloading or re-processing** large datasets.  
   - Ensures **reproducibility** — you can always load the exact dataset used in previous experiments.  
   - Works with **any Hugging Face Dataset**, including custom datasets.

4. **Result**
   - `dataset_samsum` now holds the dataset in memory as a `DatasetDict` object (usually with keys like `train`, `test`, `validation`).  
   - You can access individual splits:
     ```python
     dataset_samsum['train']   # training split
     dataset_samsum['test']    # test split
     ```

---

### ✅ In Short
`load_from_disk` restores a **saved dataset** into memory, ready for **training or evaluation**, without repeating preprocessing or downloads.


In [ ]:
from datasets import load_from_disk
dataset_samsum = load_from_disk('samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
split_lengths = [len(dataset_samsum[i])for i in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum["test"][2]["dialogue"])

print("\nSummary:")

print(dataset_samsum["test"][2]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Lenny: Babe, can you help me with something?
Bob: Sure, what's up?
Lenny: Which one should I pick?
Bob: Send me photos
Lenny:  <file_photo>
Lenny:  <file_photo>
Lenny:  <file_photo>
Bob: I like the first ones best
Lenny: But I already have purple trousers. Does it make sense to have two pairs?
Bob: I have four black pairs :D :D
Lenny: yeah, but shouldn't I pick a different color?
Bob: what matters is what you'll give you the most outfit options
Lenny: So I guess I'll buy the first or the third pair then
Bob: Pick the best quality then
Lenny: ur right, thx
Bob: no prob :)

Summary:
Lenny can't decide which trousers to buy. Bob advised Lenny on that topic. Lenny goes with Bob's advice to pick the trousers that are of best quality.


### 📘 Preparing Data for Seq2Seq Models

1. **Purpose**
   - Sequence-to-sequence (Seq2Seq) models like **BART, T5, Pegasus** require **numerical input** instead of raw text.  
   - The process converts both **input text** (e.g., dialogue) and **target text** (e.g., summary) into **token IDs** that the model can process.

2. **Example Input**
```python
{
    'dialogue': "Hi! How are you?",
    'summary': "The speaker is asking how the other person is."
}
### 📘 Tokenization for Seq2Seq Models

- **Tokenizer**: Hugging Face `AutoTokenizer`  
  Converts **words/subwords** into **numerical token IDs** that the model can process.

- **Converted Form Example**
```python
{
    'input_ids': [123, 456, 789, ...],        # Token IDs representing the input text
    'attention_mask': [1, 1, 1, ...],         # 1 = token to attend to, 0 = padding
    'labels': [321, 654, 987, ...]            # Token IDs representing the target summary
}


### 📘 Input and Target Encodings for Seq2Seq Models

1. **Input Encodings**
   - Generated using the tokenizer on the input text (e.g., dialogue):
     ```python
     input_encodings = tokenizer(example_batch['dialogue'], ...)
     ```
   - Converts text into:
     - **`input_ids`** → list of token IDs representing the input text  
     - **`attention_mask`** → list of 1s/0s to indicate which tokens the model should attend to (ignores padding tokens)

2. **Switching to Target Mode**
   - Using `tokenizer.as_target_tokenizer()` switches the tokenizer to **target mode**.  
   - Ensures **special tokens** (like `<pad>`, `<eos>`) are correctly applied for **decoder inputs** (e.g., summaries).  
   - Required for models like **BART** and **T5**.  
   - Not all Seq2Seq models need this step.

3. **Target Encodings**
   - Encodes the **summary text** into token IDs:
     - **`input_ids`** → encoder input  
     - **`attention_mask`** → tells encoder which tokens to use/ignore  
     - **`labels`** → decoder target sequence (summary token IDs)

4. **Return Dictionary Example**
```python
{
    'input_ids': [...],        # Encoder input token IDs
    'attention_mask': [...],   # Encoder attention mask
    'labels': [...]            # Decoder target token IDs (summary)
}


In [ ]:
def convert_batch_to_features(batch):
    # Encode the input (dialogue)
    input_encodings = tokenizer(
        batch['dialogue'],
        max_length=1024,
        truncation=True
    )

       # Encode the target (summary)
    target_encodings = tokenizer(
        batch['summary'],
        max_length=128,
        truncation=True
    )

    # Return the three tensors the model needs
    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


map is an inbuilt feature of the 🤗 datasets library.It’s similar to Python’s map(), but designed for datasets.

It applies a function (like your convert_batch_to_features) to each example or batch of examples in the dataset.

In [ ]:
dataset_samsum_pt = dataset_samsum.map(convert_batch_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
dataset_samsum_pt['test']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 819
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [ ]:
!pip install --upgrade transformers

### 📘 Explanation of `TrainingArguments` for Pegasus Fine-Tuning

1. **Purpose**
   - `TrainingArguments` is a configuration object in Hugging Face Transformers.  
   - It tells the `Trainer` **how to train the model**, including batch sizes, number of epochs, evaluation strategy, logging, and saving behavior.

2. **Key Arguments Explained**
   - **`output_dir='pegasus-samsum'`** → Directory where the model checkpoints and outputs will be saved.  
   - **`num_train_epochs=1`** → Number of passes over the training dataset.  
   - **`warmup_steps=500`** → Number of steps to gradually increase the learning rate at the start of training.  
   - **`per_device_train_batch_size=1`** → Batch size for each device during training.  
   - **`per_device_eval_batch_size=1`** → Batch size for evaluation.  
   - **`weight_decay=0.01`** → L2 regularization to prevent overfitting.  
   - **`logging_steps=10`** → Log training metrics every 10 steps.  
   - **`eval_strategy='steps'`** → Evaluate the model every fixed number of steps.  
   - **`eval_steps=500`** → Run evaluation every 500 steps.  
   - **`save_steps=1e6`** → Save checkpoints every 1,000,000 steps (effectively disables frequent saving).  
   - **`gradient_accumulation_steps=16`** → Accumulate gradients over 16 steps before performing an optimizer step (helps simulate larger batch sizes).  
   - **`report_to="none"`** → Disable reporting to external trackers (like WandB or TensorBoard).

3. **Why It’s Useful**
   - Centralizes all **training hyperparameters** in one object.  
   - Makes the training process **reproducible and configurable**.  
   - Works seamlessly with Hugging Face `Trainer`, handling evaluation, logging, and saving automatically.

---

### ✅ In Short
This setup prepares a **Pegasus model for fine-tuning** with controlled batch sizes, learning schedule, evaluation frequency, and logging, while using gradient accumulation to handle memory constraints.


In [ ]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    eval_strategy='steps',
    eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16,
    report_to="none"
)

### 📘 Explanation of Hugging Face `Trainer` Setup

1. **Purpose**
   - `Trainer` is a high-level API in Hugging Face Transformers.  
   - It handles **training, evaluation, and checkpointing** for models with minimal boilerplate.

2. **Key Components in This Setup**
   - **`model=model_pegasus`** → The Pegasus model to be fine-tuned.  
   - **`args=trainer_args`** → Training configuration (batch size, epochs, logging, evaluation steps, etc.).  
   - **`tokenizer=tokenizer`** → Tokenizer used for encoding input and target sequences.  
   - **`data_collator=seq2seq_data_collator`** → Prepares batches for sequence-to-sequence training (pads inputs, converts to tensors, aligns labels).  
   - **`train_dataset=dataset_samsum_pt["train"]`** → Training dataset split.  
   - **`eval_dataset=dataset_samsum_pt["validation"]`** → Validation dataset split for evaluation during training.

3. **Why It’s Useful**
   - Automates the **training loop**, including forward/backward passes, gradient accumulation, and optimizer steps.  
   - Automatically handles **evaluation** and **logging**.  
   - Manages **model checkpointing** for saving intermediate or final models.  
   - Simplifies **Seq2Seq training**, especially with Hugging Face pretrained models.

---

### ✅ In Short
This setup creates a `Trainer` instance ready to **fine-tune Pegasus** on the SAMSum dataset with all necessary components (model, tokenizer, datasets, data collator, and training arguments) properly configured.


In [ ]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])

/tmp/ipython-input-3860022235.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Step,Training Loss,Validation Loss


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:4034: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=52, training_loss=2.978379854789147, metrics={'train_runtime': 724.9323, 'train_samples_per_second': 1.13, 'train_steps_per_second': 0.072, 'total_flos': 314017624350720.0, 'train_loss': 2.978379854789147, 'epoch': 1.0})


---

## 📊 Key Metrics

- **`global_step = 52`**  
  Total optimizer steps (gradient updates) completed.  

- **`training_loss = 2.978`**  
  Final average training loss after all steps.  
  (Improved compared to the previous run: 3.05 → 2.97 ✅)  

- **`metrics`**
  - **`train_runtime = 724.93s` (~12 minutes 5 seconds)**  
    Total wall-clock training time.  

  - **`train_samples_per_second = 1.13`**  
    Throughput — number of training samples processed per second.  

  - **`train_steps_per_second = 0.072`**  
    Training speed — about 1 optimizer step every ~14 seconds.  

  - **`total_flos = 3.14 × 10^14`**  
    Total floating point operations performed (a measure of computation).  

  - **`train_loss = 2.978`**  
    Same as `training_loss`.  

  - **`epoch = 1.0`**  
    Training completed **exactly one full epoch** (one full pass through the dataset).  

---

## ✅ Interpretation

- Training was **slower** in this run (12 minutes vs ~2 minutes in the previous run).  
- **Loss improved from ~3.05 → ~2.98**, showing the model is learning.  
- **Perplexity ≈ exp(2.978) ≈ 19.6** (better than the previous ~21.1).  
- Throughput was **lower** (1.13 samples/sec vs ~6 samples/sec earlier).  
  - Possible reasons: larger batch size, slower hardware, dataset loading bottlenecks, or increased logging.  

---

## 📈 Trend

| Metric                 | Previous Run | Current Run |
|-------------------------|--------------|-------------|
| Training Loss           | 3.05         | **2.98** ✅ |
| Perplexity (approx)     | 21.1         | **19.6** ✅ |
| Runtime                 | 141s (~2m21s)| **725s (~12m)** ⏱️ |
| Samples/sec             | 5.78         | **1.13** 🔻 |
| Steps/sec               | 0.36         | **0.072** 🔻 |
| Epochs Completed        | ~1           | 1.0 |

---

💡 **Summary:** The model is improving (lower loss, lower perplexity), but the training speed dropped significantly.  

---

Do you want me to also add a **“Possible Next Steps”** section in this README (like tuning batch size, enabling mixed precision, or using a DataLoader with more workers) to help improve training speed?


🟢 What is Training Loss?
🔹 Training Loss

Measured on the training dataset (the data the model actually sees during learning).

Calculated at each training step / epoch.

It tells: “How well is the model doing on the data it’s practicing with?”

🔹 Validation Loss

Measured on the validation dataset (separate from training).

Calculated after each epoch or at checkpoints.

It tells: “How well is the model doing on new data it hasn’t practiced on yet?”

Helps detect overfitting (when training loss keeps going down but validation loss goes up).

🔹 Test Loss

Measured only once after training is done.

This is the “exam score” on completely unseen data.

It gives the most honest picture of real-world performance.

If you only pass eval_dataset, it will evaluate only when you call trainer.evaluate() or if evaluation_strategy is set in TrainingArguments.

Hugging Face used to include metrics in datasets.load_metric(), but now metrics are in a separate library: evaluate
.

Without it, calling load_metric("rouge") or other metrics will fail.

In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00


In [ ]:
import evaluate

rouge = evaluate.load("rouge") #get a rogue object

# Example
results = rouge.compute(predictions=["The cat is on the mat."],
                        references=["The cat sat on the mat."])
print(results)


{'rouge1': np.float64(0.8333333333333334), 'rouge2': np.float64(0.6), 'rougeL': np.float64(0.8333333333333334), 'rougeLsum': np.float64(0.8333333333333334)}


In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [ ]:
lst = [1,2,3,4,5,6]
list(generate_batch_sized_chunks(lst, 3))  # -> [[1,2,3], [4,5,6]]

[[1, 2, 3], [4, 5, 6]]

### 📘 Key Components for Evaluating a Summarization Model

- **`dataset`** → Your test data, e.g., a set of articles and their reference summaries.  

- **`metric`** → The scoring tool used to evaluate model performance, e.g., **ROUGE**, **BLEU**, etc.  

- **`model`** → The summarization model being evaluated, e.g., **PEGASUS**.  

- **`tokenizer`** → Converts text into **numerical token IDs** that the model can understand.  

- **`batch_size`** → Number of examples processed at a time during evaluation (affects memory usage and speed).  

- **`column_text`** → The column in your dataset containing the **articles/input text**.  

- **`column_summary`** → The column in your dataset containing the **reference summaries** (used for comparison).  

---

### ✅ In Short
These parameters define **how to feed the model test data, tokenize it, generate predictions, and evaluate performance** using metrics like ROUGE or BLEU.


tqdm is a Python library for showing progress bars in loops.

It’s just for visualization — it does not affect your calculations or results.

In [ ]:
!pip install tqdm


In [ ]:
from tqdm import tqdm

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [ ]:
import evaluate

rouge = evaluate.load("rouge") #get a rogue object

score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)


100%|██████████| 5/5 [00:47<00:00,  9.55s/it]


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
# Directly use the scores without accessing fmeasure or mid
rouge_dict = {rn: score[rn] for rn in rouge_names}

# Convert the dictionary to a DataFrame for easy visualization
import pandas as pd
pd.DataFrame(rouge_dict, index=[f'pegasus'])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.018157,0.0,0.018043,0.017909


**Let's save the model and toeknizer**

In [ ]:
model_pegasus.save_pretrained("pegasus-samsum-model")

In [ ]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

***Let's use the model to summarize***

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

### 📘 Summary Generation Parameters

- **`length_penalty`**  
  - Controls the length of generated summaries.  
  - `< 1` → encourages **shorter summaries**  
  - `> 1` → encourages **longer summaries**  
  - `1` → neutral (balanced length)

- **`num_beams`**  
  - Number of **beams used in beam search** during text generation.  
  - Higher number → more exhaustive search → usually **better quality summaries** but **slower generation**.  
  - Example: `num_beams=8` uses 8 beams.

- **`max_length`**  
  - Maximum number of **tokens** in the generated summary.  
  - Prevents the model from producing excessively long outputs.  
  - Example: `max_length=128` caps the summary at 128 tokens.

---

### ✅ In Short
These parameters control **how the model generates summaries**, balancing **length, quality, and computational cost**.


In [ ]:
# create a dictionary of arguments
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)




Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


### 📘 Hugging Face Summarization Pipeline

- **`pipeline("summarization")`**  
  Specifies the task to **generate text summaries** using a sequence-to-sequence model.

- **`model="pegasus-samsum-model"`**  
  - Can be a **pretrained model from Hugging Face Hub** or a **local folder** containing a fine-tuned model.  
  - The pipeline automatically loads the model along with its configuration.

- **`tokenizer=tokenizer`**  
  Converts **text to token IDs** and vice versa.  
  Must match the model to ensure correct input/output processing.

---

### ✅ In Short
This setup creates a **ready-to-use summarization pipeline**, which automatically handles **tokenization, model inference, and text generation** for summaries.


In [ ]:
from transformers import pipeline
print("\nModel Summary:")

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])


Model Summary:


Device set to use cuda:0
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .
